In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import os

In [2]:
model_path = r''+os.getcwd()+'\\jerma_model\\'
model = tf.keras.models.load_model(model_path)

In [3]:
test =  [
    "your mom laughing my ass off",
    "ster you are spee"
]

In [4]:
predictions = model.predict(test)
print(predictions)

[[0.35929132]
 [0.13850003]]
